In [1]:
import numpy as np

def create_maze():
    rows = int(input("Enter the number of rows: "))
    cols = int(input("Enter the number of columns: "))
    maze = np.zeros((rows, cols), dtype=int)
    print("Enter the maze layout (0 for empty space, 1 for wall, S for start, G for goal):")
    
    start = None
    goal = None
    for row in range(rows):
        row_data = input().strip()
        for col, cell in enumerate(row_data):
            if cell == 'S':
                start = row * cols + col  # Store start as a linear index
                maze[row, col] = 0
            elif cell == 'G':
                goal = row * cols + col  # Store goal as a linear index
                maze[row, col] = 0
            else:
                maze[row, col] = int(cell)
    
    return maze, start, goal

maze, initial_state, goal_state = create_maze()

Enter the maze layout (0 for empty space, 1 for wall, S for start, G for goal):


In [2]:
class QLearningAgent:
    def __init__(self, num_states, num_actions, learning_rate=0.1, discount_factor=0.9, exploration_prob=0.2):
        self.num_states = num_states
        self.num_actions = num_actions
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_prob = exploration_prob
        self.q_table = np.zeros((num_states, num_actions))

    def choose_action(self, state, explore=True):
        if explore and np.random.rand() < self.exploration_prob:
            return np.random.choice(self.num_actions)  
        else:
            return np.argmax(self.q_table[state])  

    def learn(self, state, action, reward, next_state):
        predicted = self.q_table[state, action]
        target = reward + self.discount_factor * np.max(self.q_table[next_state])
        self.q_table[state, action] += self.learning_rate * (target - predicted)

num_states = maze.size
num_actions = 4  # Up, Down, Left, Right

In [3]:
def get_next_state(state, action, maze):
    rows, cols = maze.shape
    row, col = divmod(state, cols)

    if action == 0 and row > 0:  # Move Up
        row -= 1
    elif action == 1 and row < rows - 1:  # Move Down
        row += 1
    elif action == 2 and col > 0:  # Move Left
        col -= 1
    elif action == 3 and col < cols - 1:  # Move Right
        col += 1

    next_state = row * cols + col
    if maze[row, col] == 1:  # If it's a wall, stay in the same place
        return state
    return next_state

In [4]:
agent = QLearningAgent(num_states, num_actions)

def train_agent(agent, num_episodes=1000):
    for episode in range(num_episodes):
        state = initial_state
        done = False
        while not done:
            action = agent.choose_action(state)
            next_state = get_next_state(state, action, maze)
            
            if next_state == goal_state:
                reward = 1  # Reached the goal
                done = True
            else:
                reward = 0  # Moved to an empty cell
            
            agent.learn(state, action, reward, next_state)
            state = next_state

train_agent(agent, num_episodes=1000)

In [5]:
def test_agent(agent):
    state = initial_state
    while state != goal_state:
        action = agent.choose_action(state, explore=False)  # Exploit learned policy
        print(f"Current State: {state}, Chosen Action: {action}")
        next_state = get_next_state(state, action, maze)
        if state == next_state:
            print("Hit a wall! Stay in the same position.")
        state = next_state
        print(f"New State: {state}")
    print("Agent reached the goal!")

test_agent(agent)

Current State: 0, Chosen Action: 1
New State: 3
Current State: 3, Chosen Action: 3
New State: 4
Current State: 4, Chosen Action: 1
New State: 7
Current State: 7, Chosen Action: 3
New State: 8
Agent reached the goal!
